# Carregamento dos Relatórios do MongoDB

## Importar Bibliotecas

In [13]:
from pymongo import MongoClient
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark import SparkContext
from datetime import datetime
from functools import reduce
from pyspark.sql.functions import sum
from matplotlib import cbook, cm
from matplotlib.colors import LightSource 
import matplotlib.pyplot as plt
from pyspark.sql.window import Window

## Carregamento das Análises Feitas com os Dados na Base de Dados para os Relatórios de Cada Empresa

As análises combinaram dados do YouTube e do mercado de ações para gerar relatórios detalhados por empresa.

### Etapas do Processo

#### Processamento do YouTube:
- Consolidamos métricas diárias (*visualizações*, *curtidas*, *comentários*).  
- Identificamos o vídeo mais relevante por empresa e por dia.

#### Processamento dos Dados de Mercado:
- Agregamos métricas diárias (*peak maximo*, *peak minimo*, *volume* e *stock final*).
- Calculamos médias gerais de *stocks* para cada empresa.

#### Integração e Análise:
- Combinamos dados por empresa e data.  
- Calculamos movimentações diárias de *stocks* e estruturamos os relatórios com métricas agregadas.

#### Armazenamento:
- Criamos *DataFrames* individuais para cada empresa.  
- Salvamos os relatórios no MongoDB.


In [14]:
spark_ = SparkSession.builder \
    .appName("Data Integration") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .config("spark.driver.memory", "6g") \
    .config("spark.executor.memory", "6g") \
    .getOrCreate()

df_youtube = spark_.read.format("mongo") \
    .option("uri", "mongodb://mongodb:27017/Final_Database.youtube").load()

df_company = spark_.read.format("mongo") \
    .option("uri", "mongodb://mongodb:27017/Final_Database.company").load()

In [15]:
companies = ['nvidia', 'dell', 'ibm', 'intel', 'microsoft', 'sony']

In [16]:
df_youtube = df_youtube.withColumn("day", F.to_date(df_youtube["publishedAt"]))

window_spec = Window.partitionBy("company", "day").orderBy(F.desc("view_count"))

df_youtube_with_rank = df_youtube.withColumn(
    "rank", F.row_number().over(window_spec)
)

df_youtube_top_video = df_youtube_with_rank.filter(F.col("rank") == 1)

df_youtube_daily = df_youtube_top_video.groupBy("company", "day").agg(
    F.sum("view_count").alias("total_views"),
    F.sum("likes").alias("total_likes"),
    F.sum("dislikes").alias("total_dislikes"),
    F.sum("comment_count").alias("total_coments"),
    F.first("title").alias("top_video")  
)
df_company = df_company.withColumn("day", F.to_date(df_company["Date"])) 
df_company_daily = df_company.groupBy("company_name", "day").agg(
    F.max("High").alias("stock_high"),
    F.min("Low").alias("stock_low"),
    F.avg("Volume").alias("Volume"),
    F.avg("Close").alias("final_stocks")  
)
df_report = df_youtube_daily.alias("youtube").join(
    df_company_daily.alias("company"),
    (F.col("youtube.company") == F.col("company.company_name")) & (F.col("youtube.day") == F.col("company.day")),
    "left"
).join(
    df_company_avg_stocks.alias("avg_stocks"),
    F.col("youtube.company") == F.col("avg_stocks.company_name"),
    "left"
)
df_report = df_report.withColumn(
    "final_stocks",
    F.coalesce(F.col("final_stocks"), F.lit(0))  
)
window_spec = Window.partitionBy("youtube.company").orderBy("youtube.day")

df_report = df_report.withColumn(
    "previous_final_stocks",
    F.coalesce(F.lag("final_stocks").over(window_spec), F.lit(0))  
)
df_report = df_report.withColumn(
    "stocks_movement",
    (F.col("final_stocks") - F.col("previous_final_stocks")).cast("float")
)
df_report = df_report.withColumn(
    "top_video",
    F.concat_ws(",", F.col("top_video"))
)
df_report = df_report.drop("company_name")  

company_dfs = {}

for company_name in companies:
    company_report = df_report.filter(F.col("youtube.company") == company_name)
    company_dfs[company_name] = company_report

for company_name, company_df in company_dfs.items():
    company_df.write.format("mongo") \
        .option("uri", f"mongodb://mongodb:27017/Relatorio.{company_name.lower()}") \
        .mode("overwrite") \
        .save()
print("Dados processados e salvos com sucesso.")

Dados processados e salvos com sucesso.
